# Univariate Modelling

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd

stored_models = [file for file in os.listdir() if file.endswith('.model.pkl')]
stored_models

In [ ]:
estimators = []
for model_file in stored_models:
    print(f'Loading {model_file}')
    model = joblib.load(model_file)
    estimators.append((
        model_file.replace('.model.pkl', ''),
        model
    ))

estimators

In [ ]:
from data import load_data

train_data, test_data = load_data()

unique_patients = test_data['p_num'].unique()
train_data = train_data[train_data['p_num'].isin(unique_patients)]
test_data = test_data[test_data['p_num'].isin(unique_patients)]

train_data.head()

## Predict for validation data and check results

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from pipelines import pipeline

train_data_transformed = pipeline.fit_transform(train_data)

X_train = train_data_transformed.drop(columns=['bg+1:00'])
y_train = train_data_transformed['bg+1:00']

base_models = [model for model in estimators]
meta_model = ElasticNet(alpha=0.1, l1_ratio=0.5, max_iter=10000)

model = StackingRegressor(estimators=base_models, final_estimator=meta_model)
model.fit(X=X_train, y=y_train)

# Prepare test results

In [ ]:
test_data = pipeline.transform(test_data)
y_pred = model.predict(test_data)

# count and replace negative values
if np.sum(y_pred < 0) > 0:
    print(f'Number of negative values: {np.sum(y_pred < 0)}')
    bg_min_train = np.min(y_train)
    print(f'Min value: {np.min(y_pred)}')
    y_pred = y_pred.apply(lambda x: bg_min_train if x < 0 else x)

test_data['bg+1:00'] = y_pred
test_data.head()

## Prepare the submission file

In [ ]:
submission = pd.DataFrame(test_data['bg+1:00'])
submission

### Save the submission file

In [ ]:
submission.to_csv(f'submission-{os.path.basename(os.getcwd())}.csv')